
Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

(дополнительно необязательная задача)на выбор взять
https://huggingface.co/datasets/sberquad
https://huggingface.co/datasets/blinoff/medical_qa_ru_data
натренировать любую модель для вопросно ответной системы
как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [ ]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 1.3 MB 64.1 MB/s 
     |████████████████████████████████| 7.6 MB 64.6 MB/s 
     |████████████████████████████████| 182 kB 81.1 MB/s 


In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'paraphrases'],
        num_rows: 1725393
    })
})

In [4]:
corpus['train']

Dataset({
    features: ['id', 'paraphrases'],
    num_rows: 1725393
})

In [5]:
corpus['train'][:1]

{'id': ['0'],
 'paraphrases': [['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
   'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем']]}

In [6]:
corpus['train']['paraphrases'][0]

['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
 'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем']

In [7]:
corpus['train']['paraphrases'][0][0]

'Птичий праздник: Что такое "куриное рождество" и кто его отмечает'

In [8]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

предобученная модель: cointegrated/rut5-base-paraphraser

 https://huggingface.co/cointegrated/rut5-base-paraphraser

In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda();
model.eval();

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(paraphrase('Каждый охотник желает знать, где сидит фазан.'))

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Все охотники хотят знать где фазан сидит.


предобученная модель: cointegrated/rubert-base-cased-dp-paraphrase-detection

https://huggingface.co/cointegrated/rubert-base-cased-dp-paraphrase-detection

In [10]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()

    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"

    return [text1, text2, p[0], p[1], paraphrase]


print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))

print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

['Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода', 0.7056231, 0.29437694, 'non']
['Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась', 0.16524374, 0.8347563, 'paraphrase']


In [11]:
def compare(text1, text2):
    """  """

    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()

    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    paraphrase = "non"

    if p[1] > 0.5:
        paraphrase = "paraphrase"

    return [text1, text2, p[0], p[1], paraphrase]

In [12]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)

In [13]:
result = []

result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
result.append(compare('Цены на нефть восстанавливаются', 'Парламент Словакии поблагодарил народы'))
result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице прекрасная погода'))
n=5
for i in range(n):
  text1 = corpus['train']['paraphrases'][i][0]
  text2 =corpus['train']['paraphrases'][i][1]
  result.append(compare(text1, text2))



col_names =  ['текст1', 'текст2', 'вероятность non-paraphrase', 'вероятность paraphrase', 'paraphrase']
df = pd.DataFrame(result, columns=col_names)
df.head(10)

,текст1,текст2,вероятность non-paraphrase,вероятность paraphrase,paraphrase
0,Сегодня на улице хорошая погода,Сегодня на улице отвратительная погода,0.705623,0.294377,non
1,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы,0.944955,0.055045,non
2,Сегодня на улице хорошая погода,Сегодня на улице прекрасная погода,0.035242,0.964758,paraphrase
3,"Птичий праздник: Что такое ""куриное рождество"" и кто его отмечает",Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем,0.157879,0.842121,paraphrase
4,В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото),"Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства",0.018209,0.981791,paraphrase
5,"Томас Бах: ""Олимпийский комитет России понес достаточное наказание""",Президент МОК Бах: Россия уже достаточно наказана за допинг,0.028516,0.971484,paraphrase
6,Россия не представила WADA данные по пробам в срок и может быть наказана,WADA: Россия не предоставила нам доступ к допинг-пробам,0.058580,0.941420,paraphrase
7,Антидопинговое агентство США призвало ВАДА отстранить российских спортсменов от соревнований,Комиссия спортсменов WADA потребовала отстранить РУСАДА,0.428033,0.571967,paraphrase
